### Multi-Model Performance Evaluation ###

In [1]:
import os
import copy
import numpy as np
import pandas as pd
from typing import List
import glob
import logging
from collections import deque

logger = logging.getLogger(__name__)

# Binary perfomance metrics
from sklearn.metrics import confusion_matrix

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
from llmt.performance import Performance, binary_performance

In [2]:
data_root = os.environ.get('DATA')
data_dir = os.path.join(data_root, 'hcp')
model_dir = os.path.join(data_dir, 'models')
model_file_list = sorted(glob.glob(os.path.join(model_dir, 'hcp_predictions_*.parquet')))
print(*model_file_list, sep='\n')
pred_col_list = ['pred_mh', 'pred_ip', 'pred_op']

/app/data/hcp/models/hcp_predictions_20250520_gpt-4o.parquet
/app/data/hcp/models/hcp_predictions_ollama_20250520_deepseek-v3_671b.parquet
/app/data/hcp/models/hcp_predictions_ollama_20250520_llama2_13b.parquet
/app/data/hcp/models/hcp_predictions_ollama_20250520_llama2_70b.parquet
/app/data/hcp/models/hcp_predictions_ollama_20250520_llama2_7b.parquet
/app/data/hcp/models/hcp_predictions_ollama_20250520_llama3_70b.parquet
/app/data/hcp/models/hcp_predictions_ollama_20250520_llama3_8b.parquet
/app/data/hcp/models/hcp_predictions_ollama_20250520_llama4_maverick.parquet


In [3]:
# Data set that we sent to the HCP team
df_file_name = 'Predictions-2025-04-24.xlsx'
df_file = os.path.join(data_dir, df_file_name)
dfxl = pd.read_excel(df_file)
# Train and test data
df_train = dfxl.loc[dfxl['dset'] == 'train']
df_test = dfxl.loc[dfxl['dset'] == 'test']
display(df_train.head(2))
print(dfxl.shape)
print(len(dfxl['id'].unique()))
print(df_test.shape)
print(df_train.shape)

,id,name,description,mental_health,inpatient,outpatient,dset,pred_mh,pred_ip,pred_op,verified_op,temperature
0,491639-77,144-Bed Hospital Facility Joint Venture in Den...,Provider of behavioral health services located...,1.0,1.0,1.0,train,1,1,1,1,0
1,10995-58,Acadia Healthcare (NAS: ACHC),Acadia Healthcare Co Inc acquires and develops...,1.0,1.0,2.0,train,1,1,0,1,0


(687, 12)
687
(500, 12)
(187, 12)


### Helper functions ###

In [22]:
# Helper functions
def flat_list(input_list: list) -> list:
    flattened_list = [item for sublist in input_list for item in sublist]
    return flattened_list 
    
def rotate_columns(data, by=1):
    col_deque = deque(list(data.columns))
    col_deque.rotate(by)
    return data[list(col_deque)]
    
# A function to combine columns
def combine_columns(data, true_col_list, pred_col_list):
    ''' Combine binary columns like [true_mh, true_ip] '''
    pf = Performance(data=data)
    true_combined = pf.combine_binary_columns(input_col_list=true_col_list).\
        drop(true_col_list, axis=1)
    pred_combined = pf.combine_binary_columns(input_col_list=pred_col_list).\
        drop(pred_col_list, axis=1)
    data_combined = data.\
        merge(true_combined, on='id', how='left').\
        merge(pred_combined, on='id', how='left')
    # Rename the columns in the output 
    return data_combined

def sort_by_model_order(data, model_order=None):
    if model_order is None:
        model_order = ['llama2_7b', 'llama2_13b', 'llama2_70b', 'llama3_8b', 'llama3_70b', 'llama4_maverick', 'deepseek-v3_671b', 'gpt-4o']
    data = data.assign(model=pd.Categorical(data['model'], categories=model_order, ordered=True))
    return data.sort_values(by='model')

### Training set performance ###
Use manual labels as ground truth

In [5]:
# Column names
true_mh = 'mental_health'
true_ip = 'inpatient'
true_op = 'outpatient'
true_mh_ip = 'mh_ip'
true_mh_op = 'mh_op'

pred_mh = 'pred_mh'
pred_ip = 'pred_ip'
pred_op = 'pred_op'
pred_mh_ip = 'pred_mh_ip'
pred_mh_op = 'pred_mh_op'

# Dictionary with true:pred column names
col_dict = {true_mh: pred_mh, true_ip:pred_ip, true_op:pred_op,
            true_mh_ip: pred_mh_ip,
            true_mh_op: pred_mh_op}

# This function uses the definitions above
def combine_mental_health_columns(data, 
                                  true_mh_col=true_mh,
                                  true_ip_col=true_ip,
                                  true_op_col=true_op,
                                  true_mh_ip_col=true_mh_ip,
                                  true_mh_op_col=true_mh_op,
                                  pred_mh_col=pred_mh,
                                  pred_ip_col=pred_ip,
                                  pred_op_col=pred_op, 
                                  pred_mh_ip_col=pred_mh_ip,
                                  pred_mh_op_col=pred_mh_op):
                                  
    ''' Combine mental health with inpatient/outpatient services columns '''
    data_ip = combine_columns(data=data, true_col_list=[true_mh_col, true_ip_col], pred_col_list=[pred_mh_col, pred_ip_col])
    data_op = combine_columns(data=data_ip, true_col_list=[true_mh_col, true_op_col], pred_col_list=[pred_mh_col, pred_op_col])
    output = data_op.\
        rename(columns={f'{true_mh_col}_{true_ip_col}': true_mh_ip_col,
                        f'{true_mh_col}_{true_op_col}': true_mh_op_col,
                        f'{pred_mh_col}_{pred_ip_col}': pred_mh_ip_col,
                        f'{pred_mh_col}_{pred_op_col}': pred_mh_op_col})
    return output

display(col_dict)

{'mental_health': 'pred_mh',
 'inpatient': 'pred_ip',
 'outpatient': 'pred_op',
 'mh_ip': 'pred_mh_ip',
 'mh_op': 'pred_mh_op'}

In [24]:
# We compare the same variable between different models
# Start with the variable

df_train_list = []
performance_var_list = []
for v, (true_col, pred_col) in enumerate(col_dict.items()):
   
    performance_model_list = []
    for m, model_file in enumerate(model_file_list):
        df_model = pd.read_parquet(model_file)
        df_train = df_model.loc[df_model['dset'] == 'train'].reset_index(drop=True)
    
        # Create the combined variable columnd
        df_train_combined = combine_mental_health_columns(data=df_train)
        pf = Performance(data=df_train_combined)
        df_train_list.append(df_train_combined)
        
        # Performance metrics for this model
        performance_dict = pf.binary_performance(true_col=true_col, pred_col=pred_col)
        performance_df = pd.DataFrame(performance_dict, index=[m]).\
            assign(model=df_train['model'].values[0])
        performance_model_list.append(rotate_columns(performance_df))
    
    performance_model = rotate_columns(pd.concat(performance_model_list, axis=0, ignore_index=True).\
        assign(category=true_col))
    performance_model = sort_by_model_order(data=performance_model).\
        reset_index(drop=True)
    performance_var_list.append(performance_model)
    display(performance_model)

train_performance_df = pd.concat(performance_var_list, axis=0, ignore_index=True)
train_output_df = pd.concat(df_train_list, axis=0, ignore_index=True)
train_output_df = sort_by_model_order(data=train_output_df)
display(train_output_df.head(2))
print(train_output_df.shape)

,category,model,p,n,tp,tn,fp,fn,recall,precision,min_precision,specificity,f_score
0,mental_health,llama2_7b,136,35,2,35,0,134,0.0147,1.0000,0.7953,1.0000,0.0290
1,mental_health,llama2_13b,136,35,82,22,13,54,0.6029,0.8632,0.7953,0.6286,0.7100
2,mental_health,llama2_70b,136,35,129,25,10,7,0.9485,0.9281,0.7953,0.7143,0.9382
3,mental_health,llama3_8b,136,35,126,21,14,10,0.9265,0.9000,0.7953,0.6000,0.9130
4,mental_health,llama3_70b,136,35,113,25,10,23,0.8309,0.9187,0.7953,0.7143,0.8726
5,mental_health,llama4_maverick,136,35,102,30,5,34,0.7500,0.9533,0.7953,0.8571,0.8395
6,mental_health,deepseek-v3_671b,136,35,89,33,2,47,0.6544,0.9780,0.7953,0.9429,0.7841
7,mental_health,gpt-4o,136,35,112,29,6,24,0.8235,0.9492,0.7953,0.8286,0.8819


,category,model,p,n,tp,tn,fp,fn,recall,precision,min_precision,specificity,f_score
0,inpatient,llama2_7b,69,113,69,1,112,0,1.0000,0.3812,0.3791,0.0088,0.5520
1,inpatient,llama2_13b,69,113,63,19,94,6,0.9130,0.4013,0.3791,0.1681,0.5575
2,inpatient,llama2_70b,69,113,69,26,87,0,1.0000,0.4423,0.3791,0.2301,0.6133
3,inpatient,llama3_8b,69,113,69,39,74,0,1.0000,0.4825,0.3791,0.3451,0.6509
4,inpatient,llama3_70b,69,113,69,99,14,0,1.0000,0.8313,0.3791,0.8761,0.9079
5,inpatient,llama4_maverick,69,113,64,109,4,5,0.9275,0.9412,0.3791,0.9646,0.9343
6,inpatient,deepseek-v3_671b,69,113,64,107,6,5,0.9275,0.9143,0.3791,0.9469,0.9209
7,inpatient,gpt-4o,69,113,68,103,10,1,0.9855,0.8718,0.3791,0.9115,0.9252


,category,model,p,n,tp,tn,fp,fn,recall,precision,min_precision,specificity,f_score
0,outpatient,llama2_7b,94,42,94,0,42,0,1.0000,0.6912,0.6912,0.0000,0.8174
1,outpatient,llama2_13b,94,42,93,0,42,1,0.9894,0.6889,0.6912,0.0000,0.8122
2,outpatient,llama2_70b,94,42,94,0,42,0,1.0000,0.6912,0.6912,0.0000,0.8174
3,outpatient,llama3_8b,94,42,94,10,32,0,1.0000,0.7460,0.6912,0.2381,0.8545
4,outpatient,llama3_70b,94,42,78,28,14,16,0.8298,0.8478,0.6912,0.6667,0.8387
5,outpatient,llama4_maverick,94,42,74,31,11,20,0.7872,0.8706,0.6912,0.7381,0.8268
6,outpatient,deepseek-v3_671b,94,42,66,33,9,28,0.7021,0.8800,0.6912,0.7857,0.7811
7,outpatient,gpt-4o,94,42,77,31,11,17,0.8191,0.8750,0.6912,0.7381,0.8462


,category,model,p,n,tp,tn,fp,fn,recall,precision,min_precision,specificity,f_score
0,mh_ip,llama2_7b,62,104,0,102,2,62,0.0000,0.0000,0.3735,0.9808,0.0000
1,mh_ip,llama2_13b,62,104,31,58,46,31,0.5000,0.4026,0.3735,0.5577,0.4460
2,mh_ip,llama2_70b,62,104,62,38,66,0,1.0000,0.4844,0.3735,0.3654,0.6526
3,mh_ip,llama3_8b,62,104,60,50,54,2,0.9677,0.5263,0.3735,0.4808,0.6818
4,mh_ip,llama3_70b,62,104,61,88,16,1,0.9839,0.7922,0.3735,0.8462,0.8777
5,mh_ip,llama4_maverick,62,104,55,98,6,7,0.8871,0.9016,0.3735,0.9423,0.8943
6,mh_ip,deepseek-v3_671b,62,104,49,100,4,13,0.7903,0.9245,0.3735,0.9615,0.8522
7,mh_ip,gpt-4o,62,104,60,92,12,2,0.9677,0.8333,0.3735,0.8846,0.8955


,category,model,p,n,tp,tn,fp,fn,recall,precision,min_precision,specificity,f_score
0,mh_op,llama2_7b,87,40,2,40,0,85,0.0230,1.0000,0.685,1.000,0.0449
1,mh_op,llama2_13b,87,40,52,27,13,35,0.5977,0.8000,0.685,0.675,0.6842
2,mh_op,llama2_70b,87,40,86,25,15,1,0.9885,0.8515,0.685,0.625,0.9149
3,mh_op,llama3_8b,87,40,84,24,16,3,0.9655,0.8400,0.685,0.600,0.8984
4,mh_op,llama3_70b,87,40,73,31,9,14,0.8391,0.8902,0.685,0.775,0.8639
5,mh_op,llama4_maverick,87,40,66,34,6,21,0.7586,0.9167,0.685,0.850,0.8302
6,mh_op,deepseek-v3_671b,87,40,56,36,4,31,0.6437,0.9333,0.685,0.900,0.7619
7,mh_op,gpt-4o,87,40,72,33,7,15,0.8276,0.9114,0.685,0.825,0.8675


,id,name,description,mental_health,inpatient,outpatient,dset,pred_mh,pred_ip,pred_op,verified_op,model,mh_ip,pred_mh_ip,mh_op,pred_mh_op
2244,97840-81,Vheda Health,Developer of an integrated healthcare platform...,0.0,0.0,0.0,train,0,1,1,1,llama2_7b,0.0,0,0.0,0
878,227094-31,Clear Child Psychology,Provider of assessment and long-term clinical ...,2.0,0.0,1.0,train,0,1,1,1,llama2_7b,NaN,0,NaN,0


(7480, 16)


In [7]:
# Save the results
performance_name = 'performance-trainset-8models-2025-06-22.csv'
performance_file = os.path.join(data_dir, performance_name)
train_performance_df.to_csv(performance_file)

output_name = 'predictions-trainset-8models-2025-06-22.csv'
output_file = os.path.join(data_dir, output_name)
train_output_df.to_csv(output_file)

### Testset set performance ###
Use model labels as ground truth and compare with other models

In [8]:
# Ground truth model file
print(*model_file_list, sep='\n')
ground_truth_model = 'gpt-4o'
ground_truth_model_file = [file for file in model_file_list if ground_truth_model in file][0]
model_file_list_test = [file for file in model_file_list if ground_truth_model not in file]
ground_truth_columns = ['id', 'dset', 'model', 'pred_mh', 'pred_ip', 'pred_op']

print()
print(ground_truth_model_file)
df_gt = pd.read_parquet(ground_truth_model_file, columns=ground_truth_columns)
df_gt_test = df_gt.loc[df_gt['dset'] == 'test'].reset_index(drop=True)

# Let's rename the columns that we want to use as the ground truth
df_gt_test = df_gt_test.rename(columns={'model': 'reference',
                                        'pred_mh': 'mental_health',
                                        'pred_ip': 'inpatient',
                                        'pred_op': 'outpatient'}).\
    drop('dset', axis=1)
display(df_gt_test.head(2))

/app/data/hcp/models/hcp_predictions_20250520_gpt-4o.parquet
/app/data/hcp/models/hcp_predictions_ollama_20250520_deepseek-v3_671b.parquet
/app/data/hcp/models/hcp_predictions_ollama_20250520_llama2_13b.parquet
/app/data/hcp/models/hcp_predictions_ollama_20250520_llama2_70b.parquet
/app/data/hcp/models/hcp_predictions_ollama_20250520_llama2_7b.parquet
/app/data/hcp/models/hcp_predictions_ollama_20250520_llama3_70b.parquet
/app/data/hcp/models/hcp_predictions_ollama_20250520_llama3_8b.parquet
/app/data/hcp/models/hcp_predictions_ollama_20250520_llama4_maverick.parquet

/app/data/hcp/models/hcp_predictions_20250520_gpt-4o.parquet


,id,reference,mental_health,inpatient,outpatient
0,99842-23,gpt-4o,1,0,1
1,99155-98,gpt-4o,1,0,1


In [27]:
df_test_list = []
performance_var_list = []
for v, (true_col, pred_col) in enumerate(col_dict.items()):
    
    performance_model_list = []
    for m, model_file in enumerate(model_file_list):
        df_model = pd.read_parquet(model_file)
        df_test = df_model.loc[df_model['dset'] == 'test'].reset_index(drop=True).\
            drop(columns=[true_mh, true_ip, true_op]).\
            merge(df_gt_test, on='id', how='inner')
        # sort the columns so that they are in the same sequence after the merge
        diff_cols = list(set(df_test.columns).difference(df_model.columns))
        test_cols = list(df_model.columns)
        test_cols.extend(diff_cols)
        df_test = df_test[test_cols]
        
        # Create the combined variable
        df_test_combined = combine_mental_health_columns(data=df_test)
        pf = Performance(data=df_test_combined)
        df_test_list.append(df_test_combined)
        
        # Performance metrics for this model
        performance_dict = pf.binary_performance(true_col=true_col, pred_col=pred_col)
        performance_df = pd.DataFrame(performance_dict, index=[m]).\
            assign(model=df_test['model'].values[0],
                   reference=df_gt_test['reference'].values[0])
        performance_model_list.append(rotate_columns(performance_df, by=2))
    
    performance_model = rotate_columns(pd.concat(performance_model_list, axis=0, ignore_index=True).\
        assign(category=true_col))
    performance_model = sort_by_model_order(data=performance_model).\
        reset_index(drop=True)
    performance_var_list.append(performance_model)
    display(performance_model)

test_performance_df = pd.concat(performance_var_list, axis=0, ignore_index=True)
test_output_df = pd.concat(df_test_list, axis=0, ignore_index=True)
test_output_df = sort_by_model_order(data=test_output_df)
display(test_output_df.head(2))
print(test_output_df.shape)

,category,model,reference,p,n,tp,tn,fp,fn,recall,precision,min_precision,specificity,f_score
0,mental_health,llama2_7b,gpt-4o,366,134,9,134,0,357,0.0246,1.0000,0.732,1.0000,0.0480
1,mental_health,llama2_13b,gpt-4o,366,134,236,88,46,130,0.6448,0.8369,0.732,0.6567,0.7284
2,mental_health,llama2_70b,gpt-4o,366,134,364,62,72,2,0.9945,0.8349,0.732,0.4627,0.9077
3,mental_health,llama3_8b,gpt-4o,366,134,366,65,69,0,1.0000,0.8414,0.732,0.4851,0.9139
4,mental_health,llama3_70b,gpt-4o,366,134,362,107,27,4,0.9891,0.9306,0.732,0.7985,0.9589
5,mental_health,llama4_maverick,gpt-4o,366,134,326,125,9,40,0.8907,0.9731,0.732,0.9328,0.9301
6,mental_health,deepseek-v3_671b,gpt-4o,366,134,303,134,0,63,0.8279,1.0000,0.732,1.0000,0.9058
7,mental_health,gpt-4o,gpt-4o,366,134,366,134,0,0,1.0000,1.0000,0.732,1.0000,1.0000


,category,model,reference,p,n,tp,tn,fp,fn,recall,precision,min_precision,specificity,f_score
0,inpatient,llama2_7b,gpt-4o,154,346,154,2,344,0,1.0000,0.3092,0.308,0.0058,0.4724
1,inpatient,llama2_13b,gpt-4o,154,346,147,29,317,7,0.9545,0.3168,0.308,0.0838,0.4757
2,inpatient,llama2_70b,gpt-4o,154,346,154,52,294,0,1.0000,0.3438,0.308,0.1503,0.5116
3,inpatient,llama3_8b,gpt-4o,154,346,154,81,265,0,1.0000,0.3675,0.308,0.2341,0.5375
4,inpatient,llama3_70b,gpt-4o,154,346,153,305,41,1,0.9935,0.7887,0.308,0.8815,0.8793
5,inpatient,llama4_maverick,gpt-4o,154,346,117,345,1,37,0.7597,0.9915,0.308,0.9971,0.8603
6,inpatient,deepseek-v3_671b,gpt-4o,154,346,120,346,0,34,0.7792,1.0000,0.308,1.0000,0.8759
7,inpatient,gpt-4o,gpt-4o,154,346,154,346,0,0,1.0000,1.0000,0.308,1.0000,1.0000


,category,model,reference,p,n,tp,tn,fp,fn,recall,precision,min_precision,specificity,f_score
0,outpatient,llama2_7b,gpt-4o,327,173,327,0,173,0,1.0000,0.6540,0.654,0.0000,0.7908
1,outpatient,llama2_13b,gpt-4o,327,173,327,0,173,0,1.0000,0.6540,0.654,0.0000,0.7908
2,outpatient,llama2_70b,gpt-4o,327,173,327,0,173,0,1.0000,0.6540,0.654,0.0000,0.7908
3,outpatient,llama3_8b,gpt-4o,327,173,327,10,163,0,1.0000,0.6673,0.654,0.0578,0.8005
4,outpatient,llama3_70b,gpt-4o,327,173,317,142,31,10,0.9694,0.9109,0.654,0.8208,0.9393
5,outpatient,llama4_maverick,gpt-4o,327,173,298,151,22,29,0.9113,0.9313,0.654,0.8728,0.9212
6,outpatient,deepseek-v3_671b,gpt-4o,327,173,276,172,1,51,0.8440,0.9964,0.654,0.9942,0.9139
7,outpatient,gpt-4o,gpt-4o,327,173,327,173,0,0,1.0000,1.0000,0.654,1.0000,1.0000


,category,model,reference,p,n,tp,tn,fp,fn,recall,precision,min_precision,specificity,f_score
0,mh_ip,llama2_7b,gpt-4o,140,360,2,353,7,138,0.0143,0.2222,0.28,0.9806,0.0268
1,mh_ip,llama2_13b,gpt-4o,140,360,85,178,182,55,0.6071,0.3184,0.28,0.4944,0.4177
2,mh_ip,llama2_70b,gpt-4o,140,360,139,87,273,1,0.9929,0.3374,0.28,0.2417,0.5036
3,mh_ip,llama3_8b,gpt-4o,140,360,140,129,231,0,1.0000,0.3774,0.28,0.3583,0.5479
4,mh_ip,llama3_70b,gpt-4o,140,360,139,317,43,1,0.9929,0.7637,0.28,0.8806,0.8634
5,mh_ip,llama4_maverick,gpt-4o,140,360,110,358,2,30,0.7857,0.9821,0.28,0.9944,0.8730
6,mh_ip,deepseek-v3_671b,gpt-4o,140,360,107,360,0,33,0.7643,1.0000,0.28,1.0000,0.8664
7,mh_ip,gpt-4o,gpt-4o,140,360,140,360,0,0,1.0000,1.0000,0.28,1.0000,1.0000


,category,model,reference,p,n,tp,tn,fp,fn,recall,precision,min_precision,specificity,f_score
0,mh_op,llama2_7b,gpt-4o,310,190,8,189,1,302,0.0258,0.8889,0.62,0.9947,0.0502
1,mh_op,llama2_13b,gpt-4o,310,190,196,104,86,114,0.6323,0.6950,0.62,0.5474,0.6622
2,mh_op,llama2_70b,gpt-4o,310,190,308,62,128,2,0.9935,0.7064,0.62,0.3263,0.8257
3,mh_op,llama3_8b,gpt-4o,310,190,310,66,124,0,1.0000,0.7143,0.62,0.3474,0.8333
4,mh_op,llama3_70b,gpt-4o,310,190,299,159,31,11,0.9645,0.9061,0.62,0.8368,0.9344
5,mh_op,llama4_maverick,gpt-4o,310,190,271,179,11,39,0.8742,0.9610,0.62,0.9421,0.9155
6,mh_op,deepseek-v3_671b,gpt-4o,310,190,244,189,1,66,0.7871,0.9959,0.62,0.9947,0.8793
7,mh_op,gpt-4o,gpt-4o,310,190,310,190,0,0,1.0000,1.0000,0.62,1.0000,1.0000


,id,name,description,mental_health,inpatient,outpatient,dset,pred_mh,pred_ip,pred_op,verified_op,model,reference,mh_ip,pred_mh_ip,mh_op,pred_mh_op
6000,99842-23,Seminole Behavioral Healthcare,Provider of behavioral healthcare services. Th...,1,0,1,test,0,1,1,1,llama2_7b,gpt-4o,0,0,1,0
2327,238201-48,Disc Village,The company primarily operates in the Human Ca...,0,0,0,test,0,1,1,0,llama2_7b,gpt-4o,0,0,0,0


(20000, 17)


In [10]:
# Save the results
performance_name = 'performance-testset-8models-2025-06-22.csv'
performance_file = os.path.join(data_dir, performance_name)
test_performance_df.to_csv(performance_file)

output_name = 'predictions-testset-8models-2025-06-22.csv'
output_file = os.path.join(data_dir, output_name)
test_output_df.to_csv(output_file)